<center><h1>Khatri_Surbhi_HW7</h1></center>

Name: Surbhi Khatri
<br>
Github Username: surbhikhatri
<br>
USC ID: 9893833133

## 1. Multi-class and Multi-Label Classification Using Support Vector Machines

Import packages

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import math

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import hamming_loss
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

### (a) Download the Anuran Calls (MFCCs) Data Set

In [50]:
df = pd.read_csv("./../data/AnuranCalls/Frogs_MFCCs.csv")
# choose 70% randomly
x_train, x_test, y_train, y_test = train_test_split(df[df.columns[:22]], df[["Family","Genus","Species"]], test_size=0.3)

### (b) Train a classifier for each label

#### (i) Research

Exact match is used to measure the accuracy of the classifier by calculating the % of records for which  the classifier predicts the label correctly. i.e. It measures the True Positives.

Hamming loss uses the % of labels that are predicted incorrectly. It calculates the average calculating the fraction of incorrectly records, by summing up all the records where the label is incorrectly predicted and divide it by total records in the dataset.


Difference btw exact match and Hamming loss is in how they judge the model's performance. Exact match only cares about cases where every single predicted label is right. On the other hand, Hamming loss looks at how many labels were predicted wrong on average for all instances. Hamming loss is a bit more forgiving because it considers cases where some labels were right even if not all of them were correct. So, it gives a more detailed picture of how well the classifier is doing.

#### (ii) Train a SVM for each of the labels

In [51]:
gammas = [.01, 0.1, .5]
c = [1e-4,1e-3, 1e-2, 1e-1, 1,1e1,1e2,1e3,1e4]
# selecting gamma and c as taught in class

params_ = {'svc__C': c, 'svc__gamma': gammas[:2]}
svc = svm.SVC(kernel='rbf', max_iter=10000)

svm_model = make_pipeline(StandardScaler(), svc)
# use cv = 10 to tune hyperparameters
grid_search = GridSearchCV(svm_model, params_, cv=10)


In [52]:
# for family
grid_search.fit(x_train, y_train["Family"])
scv_family = grid_search.best_estimator_.fit(x_train, y_train["Family"])
print("Score for Family ", scv_family.score(x_test, y_test["Family"]))

Score for Family  0.9916628068550255


In [53]:
# predict alpha and gamma
scv_family_pred = scv_family.predict(x_test)
print("best c and gama:", grid_search.best_params_["svc__C"], grid_search.best_params_["svc__gamma"])

best c and gama: 10.0 0.1


In [54]:
# for genus
grid_search = GridSearchCV(svm_model, params_, cv=10)
grid_search.fit(x_train, y_train["Genus"])
scv_genus = grid_search.best_estimator_.fit(x_train, y_train["Genus"])
print("Score for Genus ", scv_genus.score(x_test, y_test["Genus"]))
scv_genus_pred = scv_genus.predict(x_test)

Score for Genus  0.9874942102825383


In [55]:
grid_search  = GridSearchCV(svm_model, params_, cv=10, n_jobs=-1)
grid_search.fit(x_train, y_train["Species"])
scv_species = grid_search.best_estimator_.fit(x_train, y_train["Species"])
print("Score for Species ", scv_species.score(x_test, y_test["Species"]))
scv_species_pred = scv_species.predict(x_test)

Score for Species  0.9902732746641963


In [56]:
# b (i) use hamming loss,,
hl_1 = hamming_loss(y_test["Family"], scv_family_pred)
hl_2 = hamming_loss(y_test["Genus"], scv_genus_pred)
hl_3 = hamming_loss(y_test["Species"], scv_species_pred)


print("Family Hamming loss ",  hl_1)
print("Genus Hamming loss ", hl_2)
print("Species Hamming loss ", hl_3)

print("average hamming loss = (sum of all)/n = ", (hl_1 + hl_2 + hl_3)/3)

predicted_df = pd.DataFrame({"Family": scv_family_pred, "Genus": scv_genus_pred, "Species": scv_species_pred})
print("Exact match ", (y_test.values==predicted_df.values).sum()/(len(y_test)*3))

Family Hamming loss  0.008337193144974525
Genus Hamming loss  0.012505789717461788
Species Hamming loss  0.009726725335803613
average hamming loss = (sum of all)/n =  0.010189902732746642
Exact match  0.9898100972672533


#### (iii) Repeat 1(b)ii with L1-penalized SVMs

In [57]:
# Repeat 1(b)ii with L1-penalized SVMs.3 Remember to standardize4the at-tributes. Determine the weight of the SVM penalty using 10 fold cross vali-dation.

params_ = {'linearsvc__C': [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4]}
cv = KFold(n_splits=10, shuffle=True)
model = make_pipeline(StandardScaler(), LinearSVC(penalty='l1', dual=False, max_iter=10000))

In [58]:
# for family
grid_search = GridSearchCV(model, param_grid=params_, cv=cv, scoring='accuracy')
grid_search.fit(x_train, y_train["Family"])
family_l1 = grid_search.best_estimator_.fit(x_train, y_train["Family"])
print("Score for Family ", family_l1.score(x_test, y_test["Family"]))

Score for Family  0.9351551644279759


In [59]:
# for enus
grid_search = GridSearchCV(model, param_grid=params_, cv=cv, scoring='accuracy')
grid_search.fit(x_train, y_train["Genus"])
genus_l1 = grid_search.best_estimator_.fit(x_train, y_train["Genus"])
print("Score for Genus ", genus_l1.score(x_test, y_test["Genus"]))

Score for Genus  0.9495136637332098


In [60]:
# for Species
grid_search = GridSearchCV(model, param_grid=params_, cv=cv, scoring='accuracy')
grid_search.fit(x_train, y_train["Species"])
species_l1 = grid_search.best_estimator_.fit(x_train, y_train["Species"])
print("Score for Species ", species_l1.score(x_test, y_test["Species"]))

Score for Species  0.9573876794812413


In [61]:
family_l1_pred = family_l1.predict(x_test)
genus_l1_pred = genus_l1.predict(x_test)
species_l1_pred = species_l1.predict(x_test)

# b (i) use hamming loss,,
hl_1 = hamming_loss(y_test["Family"], family_l1_pred)
hl_2 = hamming_loss(y_test["Genus"], genus_l1_pred)
hl_3 = hamming_loss(y_test["Species"], species_l1_pred)


print("Family Hamming loss ",  hl_1)
print("Genus Hamming loss ", hl_2)
print("Species Hamming loss ", hl_3)

print("average hamming loss = (sum of all)/n = ", (hl_1 + hl_2 + hl_3)/3)

predicted_df = pd.DataFrame({"Family": family_l1_pred, "Genus": genus_l1_pred, "Species": species_l1_pred})
print("Exact match ", (y_test.values==predicted_df.values).sum()/(len(y_test)*3))


Family Hamming loss  0.06484483557202408
Genus Hamming loss  0.05048633626679018
Species Hamming loss  0.042612320518758684
average hamming loss = (sum of all)/n =  0.05264783078585764
Exact match  0.9473521692141423


#### (iv) Repeat 1(b)iii by using SMOTE or any other method for imbalance

In [62]:
# initialize smote
params_ = {'linearsvc__C': c}
smote = SMOTE()
pipeline = Pipeline([
    ('standardise', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('linearsvc', LinearSVC(penalty='l1', dual=False, max_iter=10000))
])


In [63]:
cv = KFold(n_splits=10, shuffle=True)
grid_search = GridSearchCV(pipeline, param_grid=params_, cv=cv, scoring='accuracy')
grid_search.fit(x_train, y_train["Family"])
family_smote = grid_search.best_estimator_.fit(x_train, y_train["Family"])
print("Score for Family ", family_smote.score(x_test, y_test["Family"]))

Score for Family  0.9087540528022232


In [64]:
grid_search = GridSearchCV(pipeline, param_grid=params_, cv=cv, scoring='accuracy')
grid_search.fit(x_train, y_train["Genus"])
genus_smote = grid_search.best_estimator_.fit(x_train, y_train["Genus"])
print("Score for Genus ", genus_smote.score(x_test, y_test["Genus"]))

Score for Genus  0.9022695692450209


In [66]:
grid_search = GridSearchCV(pipeline, param_grid=params_, cv=cv, scoring='accuracy')
grid_search.fit(x_train, y_train["Species"])
species_smote = grid_search.best_estimator_.fit(x_train, y_train["Species"])
print("Score for Species ", species_smote.score(x_test, y_test["Species"]))

Score for Species  0.9550717924965262


In [67]:
family_smote_pred = family_smote.predict(x_test)
genus_smote_pred = genus_smote.predict(x_test)
species_smote_pred = species_smote.predict(x_test)

# b (i) use hamming loss,,
hl_1 = hamming_loss(y_test["Family"], family_smote_pred)
hl_2 = hamming_loss(y_test["Genus"], genus_smote_pred)
hl_3 = hamming_loss(y_test["Species"], species_smote_pred)


print("Family Hamming loss ",  hl_1)
print("Genus Hamming loss ", hl_2)
print("Species Hamming loss ", hl_3)

print("average hamming loss = (sum of all)/n = ", (hl_1 + hl_2 + hl_3)/3)

predicted_df = pd.DataFrame({"Family": family_smote_pred, "Genus": genus_smote_pred, "Species": species_smote_pred})
print("Exact match ", (y_test.values==predicted_df.values).sum()/(len(y_test)*3))


Family Hamming loss  0.09124594719777675
Genus Hamming loss  0.09773043075497916
Species Hamming loss  0.04492820750347383
average hamming loss = (sum of all)/n =  0.07796819515207658
Exact match  0.9220318048479235


Summary:

* For this problem, Gaussian classifier gives the best result
* SMOTE does not perform significantly better than others classifiers, errors are comparable and takes a lot tof time in training
* LinearSVC with L1 is faster to train


## 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set

K-Means Clustering on a Multi-Class and Multi-Label Data Set
Monte-Carlo Simulation: Perform the following procedures 50 times, and report
the average and standard deviation of the 50 Hamming Distances that you calculate

### (a) Use k-means clustering

In [40]:
# using Silhouettes for calculating best k and then apply monte carlo simulation

k_arr = []
kmeans_arr = []
monte_carlo_arr = []

# apply monte-carlo simulation 50 times
for i in range(50):
    # exclude the decision columns
    x = df[df.columns[:22]]
    silhouette_arr = []
    max_sil = 0
    best_k = -1
    best_kmeans = -1
    for k in range(2, 51):
        kmeans = KMeans(n_clusters=k, init='random').fit(x)
        cur_sil = silhouette_score(x, kmeans.labels_)
        if cur_sil > max_sil:
            max_sil = cur_sil
            best_k = k
            best_kmeans = kmeans

    monte_carlo_arr.append(best_kmeans)
    k_arr.append(best_k)
    print("Best k for iteration ", i, ": ", best_k)


Best k for iteration  0 :  4
Best k for iteration  1 :  4
Best k for iteration  2 :  4
Best k for iteration  3 :  4
Best k for iteration  4 :  4
Best k for iteration  5 :  4
Best k for iteration  6 :  4
Best k for iteration  7 :  4
Best k for iteration  8 :  4
Best k for iteration  9 :  4
Best k for iteration  10 :  4
Best k for iteration  11 :  4
Best k for iteration  12 :  4
Best k for iteration  13 :  4
Best k for iteration  14 :  4
Best k for iteration  15 :  4
Best k for iteration  16 :  4
Best k for iteration  17 :  4
Best k for iteration  18 :  4
Best k for iteration  19 :  4
Best k for iteration  20 :  4
Best k for iteration  21 :  4
Best k for iteration  22 :  4
Best k for iteration  23 :  4
Best k for iteration  24 :  4
Best k for iteration  25 :  4
Best k for iteration  26 :  4
Best k for iteration  27 :  4
Best k for iteration  28 :  4
Best k for iteration  29 :  4
Best k for iteration  30 :  4
Best k for iteration  31 :  4
Best k for iteration  32 :  4
Best k for iteration

### (b) Determine which family is the majority

In [104]:
monte_carlo_cluster_arr = []
from sklearn.preprocessing import LabelEncoder
df['Family'] = pd.Categorical(df['Family'])
df['Genus'] = pd.Categorical(df['Genus'])
df['Species'] = pd.Categorical(df['Species'])
true_labels_f = df['Family'].cat.codes
true_labels_g = df['Genus'].cat.codes
true_labels_s = df['Species'].cat.codes

def calculate_majority_label(l, j, cur_label, c, pos):
    idx = np.where(l == j)[0]
    cluster_labels = true_labels_f[idx]
    major = df[cur_label].cat.categories[np.bincount(cluster_labels).argmax()]
    c[j][pos] = major


for i in range(50):
    clusters = [["" for x in range(3)] for y in range(k_arr[i])]
    kmeans = monte_carlo_arr[i]
    print("itr " + str(i))
    labels = kmeans.labels_

    for j in range(k_arr[i]):
        calculate_majority_label(labels, j, 'Family', clusters, 0)
        calculate_majority_label(labels, j, 'Genus', clusters, 1)
        calculate_majority_label(labels, j, 'Species', clusters, 2)
        print("cluster " + str(j) + " Family Majority: " + clusters[j][0] + " Genus Majority: " + clusters[j][1] + " Species Majority: " + clusters[j][2])
    monte_carlo_cluster_arr.append(clusters)



itr 0
cluster 0 Family Majority: Leptodactylidae Genus Majority: Hypsiboas Species Majority: HylaMinuta
cluster 1 Family Majority: Hylidae Genus Majority: Dendropsophus Species Majority: Ameeregatrivittata
cluster 2 Family Majority: Dendrobatidae Genus Majority: Ameerega Species Majority: AdenomeraHylaedactylus
cluster 3 Family Majority: Hylidae Genus Majority: Dendropsophus Species Majority: Ameeregatrivittata
itr 1
cluster 0 Family Majority: Hylidae Genus Majority: Dendropsophus Species Majority: Ameeregatrivittata
cluster 1 Family Majority: Leptodactylidae Genus Majority: Hypsiboas Species Majority: HylaMinuta
cluster 2 Family Majority: Hylidae Genus Majority: Dendropsophus Species Majority: Ameeregatrivittata
cluster 3 Family Majority: Dendrobatidae Genus Majority: Ameerega Species Majority: AdenomeraHylaedactylus
itr 2
cluster 0 Family Majority: Dendrobatidae Genus Majority: Ameerega Species Majority: AdenomeraHylaedactylus
cluster 1 Family Majority: Hylidae Genus Majority: Dendro

### (c) Calculate the average Hamming distance, Hamming score, and Hamming loss

In [89]:
df['Family'] = pd.Categorical(df['Family'])
df['Genus'] = pd.Categorical(df['Genus'])
df['Species'] = pd.Categorical(df['Species'])
true_labels = pd.concat([df['Family'], df['Genus'], df['Species']], axis=1)

hamming_distances = []
hamming_scores = []
hamming_losses = []

def calculate_hamming_measures(d, l, ml):
    loss = d.sum().sum()/len(l)
    score_ = 1 - loss
    distance = (l != ml).sum(axis=1).mean()

    hamming_distances.append(distance)
    hamming_scores.append(score_)
    hamming_losses.append(loss)


for i in range (50):
    majority_label_triplets = monte_carlo_cluster_arr[i]
    labels = monte_carlo_arr[i].labels_
    for j in range(k_arr[i]):
        majority_label_triplet = majority_label_triplets[j]

        idx = np.where(labels == j)[0]
        c_labels = true_labels.iloc[idx]
        diff =  c_labels != majority_label_triplet

        calculate_hamming_measures(diff, c_labels, majority_label_triplet)

print("Avg Hamming Distance:", np.mean(hamming_distances))
print("Avg Hamming Score:", np.mean(hamming_scores))
print("Avg Hamming Loss:", np.mean(hamming_losses))


Avg Hamming Distance: 0.8611181973654526
Avg Hamming Score: 0.13888180263454752
Avg Hamming Loss: 0.8611181973654526


## 3. ISLR 12.6.2

Pls find hw7mlds pdf in the main directory for theory solution